In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
cicero_df = pd.read_csv('../data/cicero_df.csv')
cicero_df.head()

,title,total_clausulae,total_excluded,abbrev_excluded,bracket_excluded,short_excluded,cretic-trochee (-u--x),cretic-trochee 1 res (uuu--x),cretic-trochee 1 res (-uuu-x),cretic-trochee 1 res (-u-uux),...,hypodochmiac 1 res (uuu-ux),hypodochmiac 1 res (-uuuux),spondaic (---x),heroic (-uu-x),first paeon (-uux),choriamb trochee (-uu--x),short sequence (uuuuux),total_artistic,misc_clausulae,percent_clausulae
0,academica,228,8,0,5,2,32.0,5.0,11.0,7.0,...,1.0,1.0,20.0,7.0,7.0,4.0,1.0,217.0,3.0,1.0
1,arati phaenomena,250,19,0,19,0,6.0,0.0,3.0,0.0,...,0.0,0.0,9.0,185.0,8.0,2.0,0.0,231.0,0.0,1.0
2,arati prognotica,9,0,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,1.0
3,brutus,1655,146,74,12,53,210.0,28.0,67.0,18.0,...,2.0,1.0,95.0,13.0,31.0,4.0,3.0,1482.0,27.0,1.0
4,carmina fragmenta,125,0,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,8.0,54.0,3.0,3.0,0.0,124.0,1.0,1.0


In [3]:
cicero_speeches = cicero_df.drop(cicero_df.index[:8])
cicero_speeches = cicero_speeches.drop(cicero_df.index[9:11])
cicero_speeches = cicero_speeches.drop(cicero_df.index[12:14])
cicero_speeches = cicero_speeches.drop(cicero_df.index[15:24])
cicero_speeches = cicero_speeches.drop(cicero_df.index[25:54])
cicero_speeches = cicero_speeches.drop(cicero_df.index[112])
cicero_speeches = cicero_speeches.drop(cicero_df.index[56:60])
cicero_speeches = cicero_speeches.drop(cicero_df.index[64:77])
cicero_speeches = cicero_speeches.drop(cicero_df.index[78:93])
cicero_speeches = cicero_speeches.drop(cicero_df.index[118:])

In [4]:
cicero_speeches['title']

8                        de domo sua
11            de haruspicum responso
14                   de lege agraria
24        de provinciis consularibus
54                      in caecilium
55                      in catilinam
60                        in pisonem
61                    in sallustinum
62                       in vatinium
63                         in verrem
77                       philippicae
93           post reditum ad populum
94            post reditum in senatu
95                        pro archia
96                         pro balbo
97                       pro caecina
98                        pro caelio
99                      pro cluentio
100                       pro flacco
101                      pro fonteio
102                 pro lege manilia
103                      pro ligario
104                     pro marcello
105                       pro milone
106                       pro murena
107                      pro plancio
108                     pro quinctio
1

In [5]:
cicero_speeches['total_unartistic'] = cicero_speeches['total_clausulae'] - cicero_speeches['total_excluded'] - cicero_speeches['total_artistic']

In [6]:
contingency_table = cicero_speeches[['title', 'total_artistic', 'total_unartistic']]

In [7]:
total_artistic = contingency_table['total_artistic'].sum()
total_unartistic = contingency_table['total_unartistic'].sum()

In [8]:
def chi_square_statistic(artistic, unartistic, p_value=False):
    obs = np.array([
        [artistic, unartistic],
        [total_artistic-artistic, total_unartistic-unartistic]
    ])
    chi2, p, _, _ = stats.chi2_contingency(obs, correction=False)
    
    if p_value:
        return p
    
    return chi2

In [9]:
contingency_table['chi2'] = contingency_table.apply(
    lambda row: chi_square_statistic(row['total_artistic'], row['total_unartistic']),
    axis=1
)
contingency_table['p'] = contingency_table.apply(
    lambda row: chi_square_statistic(row['total_artistic'], row['total_unartistic'], p_value=True),
    axis=1
)

/Users/tyler/Projects/latin-prose-rhythm/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/tyler/Projects/latin-prose-rhythm/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [10]:
print('Total Artistic: ', total_artistic)
print('Total Unartistic: ', total_unartistic)

Total Artistic:  23838.0
Total Unartistic:  471.0


In [11]:
contingency_table

,title,total_artistic,total_unartistic,chi2,p
8,de domo sua,721.0,15.0,0.040338,0.840821
11,de haruspicum responso,397.0,4.0,1.896320,0.168491
14,de lege agraria,765.0,21.0,2.304471,0.129002
24,de provinciis consularibus,252.0,2.0,1.787104,0.181279
54,in caecilium,309.0,7.0,0.129887,0.718550
55,in catilinam,641.0,11.0,0.221155,0.638161
60,in pisonem,603.0,14.0,0.366129,0.545122
61,in sallustinum,106.0,2.0,0.004194,0.948367
62,in vatinium,190.0,0.0,3.783663,0.051755
63,in verrem,6193.0,129.0,0.476508,0.490008
